In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from VMOSfunc import *

In [2]:
def spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2=False):
    output_par = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    output_par_err = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    fitted = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    residual = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])    
            popt_full_fit,pcov_full_fit = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)
            
            output_par[:,i,j]=popt_full_fit
            fitted[:,i,j]=(full_gauss_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
            residual[:,i,j] = mini_cube[:,i,j] - fitted[:,i,j]
            
            spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
            for l in range(MC_loops):
                iteration_data = np.random.normal(spectrum,error)   
                popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
                spec_parameters_MC[:,l]=popt_spec_MC
                
            spec_parameters_err = np.std(spec_parameters_MC,1)
            output_par_err[:,i,j] = spec_parameters_err
    return output_par,output_par_err,fitted,residual

def flux_map(output_par,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(output_par[1,:,:],name='amp_OIII5007'))
    hdus.append(fits.ImageHDU(output_par[2,:,:],name='amp_OIII5007_br'))
    hdus.append(fits.ImageHDU(output_par[3,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(output_par[4,:,:],name='amp_Hb1'))
    hdus.append(fits.ImageHDU(output_par[5,:,:],name='amp_Hb2'))
    hdus.append(fits.ImageHDU(output_par[6,:,:],name='amp_Fe5018_1'))
    hdus.append(fits.ImageHDU(output_par[7,:,:],name='amp_Fe5018_2'))
    hdus.append(fits.ImageHDU(output_par[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(output_par[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/19_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def flux_map_err(output_par_err,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par_err[0,:,:],name='amp_Hb_err'))
    hdus.append(fits.ImageHDU(output_par_err[1,:,:],name='amp_OIII5007_err'))
    hdus.append(fits.ImageHDU(output_par_err[2,:,:],name='amp_OIII5007_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[3,:,:],name='amp_Hb_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[4,:,:],name='amp_Hb1_err'))
    hdus.append(fits.ImageHDU(output_par_err[5,:,:],name='amp_Hb2_err'))
    hdus.append(fits.ImageHDU(output_par_err[6,:,:],name='amp_Fe5018_1_err'))
    hdus.append(fits.ImageHDU(output_par_err[7,:,:],name='amp_Fe5018_2_err'))
    hdus.append(fits.ImageHDU(output_par_err[8,:,:],name='m_err'))
    hdus.append(fits.ImageHDU(output_par_err[9,:,:],name='c_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/19_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')


In [3]:
def spectroastrometric_script(obj,z,broad2,MC_loops=100,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    fixed_param = fixed_parameters(obj)
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.19_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
 
    (output_par,output_par_err,fitted_cube,residual_cube) = spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2)
    
    store_cube('%s/%s/%s.19_arcsec_fitted_residual_minicube.fits'%(destination_path_cube,obj,obj),fitted_cube,mini_wave,residual_cube,mini_header)    
    flux_map(output_par,obj)
    flux_map_err(output_par_err,obj)
    
    (centroid_OIII_nr,centroid_OIII_br,centroid_Hb_blr_br) = light_weighted_centroid(obj)
    print ('%s.fits'%(obj))

In [4]:
z = {"HE1310-1051":0.03427,"HE1338-1423":0.04179}

objs = z.keys()
broad2= {"HE1310-1051":False,"HE1338-1423":True}

for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])

HE1310-1051.fits
HE1338-1423.fits
